# DSI-24 EEG Triggering: Concepts and Best Practices

This document provides an overview of **why** and **how** to send triggers when using the DSI-24 wireless EEG system. It covers the conceptual underpinnings of different trigger styles (tonic vs. phasic) and timing methods (immediate vs. on-flip), along with best practices for establishing a reliable serial connection and ensuring high-quality EEG event alignment.

---

## 1. Introduction

In many EEG experiments, sending **event markers** (also called *triggers*) from your behavioral task software (e.g., **PsychoPy**) to the DSI-24 amplifier is crucial for synchronizing recorded brain signals with specific paradigm events (such as stimulus onsets, condition changes, or responses). Understanding the core concepts behind **tonic** and **phasic** triggers—and how to manage them in your experimental workflow—lays a solid foundation for accurate and meaningful EEG data collection.

---

## 2. Background on DSI-24 and Triggering

### 2.1 Hardware and Serial Communication

The **DSI-24** system is a **wireless EEG headset** that can receive numeric trigger values via a **serial port** or other specialized I/O interface. Each numeric code is saved alongside the EEG data, allowing you to pinpoint the moment events occurred during later analysis.

- **Physical Connection**  
  Typically, you connect your computer (running experiment software like PsychoPy) to the DSI-24’s data acquisition software (e.g., **DSI-Streamer**) through a **USB-to-serial** adapter or a dedicated trigger interface.

- **Finding Your COM Port**  
  1. **Windows**:  
     - Open **Device Manager** (Win+X → “Device Manager”).  
     - Expand **“Ports (COM & LPT)”** and locate something like **“USB Serial Port (COMX)”**.  
  2. **Linux**:  
     - Plug in your device, then open a terminal: `dmesg | grep tty` to see the assigned port (e.g., `/dev/ttyUSB0`).  
  3. **macOS**:  
     - Open **Terminal** and run `ls /dev/tty.*` or `ls /dev/cu.*` to find entries referencing `usbserial` or similar.

Knowing your device’s port name is essential before opening and configuring the serial connection in Python.

---

## 3. Tonic vs. Phasic Triggers

Event markers (triggers) align EEG data with your task events. Two main styles are widely used in EEG experiments:

### 3.1 Tonic (Sustained) Triggers

- **Definition**  
  A **tonic** trigger is set to a nonzero value at the start of a block or event and remains at that value until it’s explicitly reset to zero.  
- **When to Use**  
  - **Long Blocks/States**: Great for extended intervals (e.g., multi-minute tasks).  
  - **Neurofeedback/Closed-Loop**: Maintains a constant marker for real-time tracking.  
- **Pros/Cons**  
  - **Pros**: Easy to visualize in raw data; simpler for long durations.  
  - **Cons**: Must remember to reset; less precise for capturing quick sub-events within a block.

### 3.2 Phasic (Brief) Triggers

- **Definition**  
  A **phasic** trigger is a short pulse—often just 10–50 ms—turned on at a precise moment (like stimulus onset) and then turned off immediately.  
- **When to Use**  
  - **Discrete Stimulus Onsets**: Common in ERP studies (e.g., P300).  
  - **Rapid-Fire Events**: Each short pulse stands alone.  
- **Pros/Cons**  
  - **Pros**: High temporal precision; reduces overlap between events.  
  - **Cons**: Not ideal for extended conditions; must ensure hardware detects short pulses.

### 3.3 Choosing the Right Approach

- **Experiment Duration**:  
  - **Tonic** for multi-second/minute intervals where a continuous marker is beneficial.  
  - **Phasic** for short-lived events that need precise timing.  
- **Analysis Goals**:  
  - **ERP or Time-Locked** studies typically prefer **phasic**.  
  - **Block-Based** or **Neurofeedback** typically prefer **tonic**.  
- **Hybrid**: Many designs combine tonic markers for broad phases with phasic pulses for precise event onsets.

---

## 4. Immediate vs. On-Flip Timing

Beyond deciding on tonic or phasic triggers, you should also consider **when** to send them in your PsychoPy drawing cycle.

1. **Immediate**:  
   - Sends the trigger the moment the code line is executed.  
   - **Use Case**: Logic-based events or key presses not tied to a screen refresh.

2. **On-Flip**:  
   - Schedules the trigger for the **next screen refresh** (`win.flip()` in PsychoPy).  
   - **Use Case**: Visual stimulus presentation requiring frame-level synchronization.

For example, an ERP study analyzing response latency might need triggers that **exactly coincide** with a new stimulus. In that case, an on-flip trigger is recommended to ensure **frame-locked accuracy**. Conversely, a logic-based marker (like “participant pressed spacebar”) can be sent immediately without referencing the display refresh.

---

## 5. Opening and Managing the Serial Connection

1. **Identify the COM Port**:  
   Find the USB-to-serial port name (e.g., `COM3`, `/dev/ttyUSB0`).
2. **Initialize and Reset**:  
   It is a best practice to **open** the serial port early in your script and **reset** the trigger line to 0 immediately, ensuring a clean starting state.
3. **Confirm Specs**:  
   Some EEG amplifiers require a minimum pulse width or specific baud rate. Check DSI-24 documentation to confirm the recommended serial settings.  

**Best Practice**: If your code is in PsychoPy, place serial initialization in a “Begin Experiment” tab or at the start of your script to ensure it’s ready by the time you start sending triggers.

---

## 6. Additional Best Practices

1. **Consistent Coding Scheme**  
   - Use a clear, documented mapping of triggers (e.g., `START_TASK = 10`, `GO_ONSET = 1`, etc.) so you can easily interpret them later.
2. **Resetting Tonic Triggers**  
   - Always remember to **reset** tonic triggers back to `0` (or another baseline) after the condition ends.
3. **Timing Requirements**  
   - Check the **minimum pulse width** for phasic triggers. Some hardware requires at least 10 ms for proper detection.  
4. **Logging and Verification**  
   - Print console messages or log to file whenever you send a trigger. This helps with debugging and ensures your DSI-24 is receiving codes as expected.
5. **Hybrid Approach**  
   - If your experiment has **long blocks** plus **precise events** within those blocks, consider combining a tonic marker for the block with **phasic** pulses for sub-events.

---

## 7. Conclusion

By understanding **tonic vs. phasic** triggers and carefully selecting **immediate vs. on-flip** timing, you can precisely align your DSI-24 EEG data with the events that matter most in your experiment. Whether you’re running a simple reaction-time task or a complex neurofeedback study, consistently applying these concepts will yield more reliable and interpretable EEG results.

For **implementation details** (e.g., full Python code snippets or PsychoPy Builder instructions), see the accompanying reference documents:
- **DSI24_EEG_Triggering_CodeSnippets_QuickReference.ipynb**  
- **DSI24_GoNoGoDemo_PsychoPy_TriggerImplementation.ipynb**
